<a href="https://colab.research.google.com/github/hemanthhariharan/RA-NEM/blob/main/Options_valuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from scipy.stats import norm, lognorm
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

## Back-calculating implied volatility

In [25]:
def func(sigma):
  s_0 = 60
  r = 0.06
  div_yield = 0
  k = ks[i]
  # T = Ts[j]
  T = Ts[i]
  # c = cs[i, j]
  c = cs[i]

  d1 = (np.log(s_0 / k) + (r - div_yield + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)

  # return -s_0 * np.exp(-div_yield * T) * norm.cdf(-d1) + k * np.exp(-r * T) * norm.cdf(-d2) - c # for put
  return s_0 * np.exp(-div_yield * T) * norm.cdf(d1) - k * np.exp(-r * T) * norm.cdf(d2) - c # for call

In [89]:
ks = np.arange(30, 90, 10)
Ts = np.full((6, ), 0.5)
# cs = np.array(
#     [
#         [0.0419, 0.0236, 0.0236],
#         [0.0236, 0.0236, 0.0236],
#         [0.0236, 0.0236, 0.0236]
#     ]
# )

cs = np.empty((6, ))

for i in range(6):
  p = (60 * np.exp(0) - 50) / 25
  cs[i] = p * euro_option_price(75, ks[i], 0.5, 0.06, 0.25) + (1 - p) * euro_option_price(50, ks[i], 0.5, 0.06, 0.4)
  # cs[i] = euro_option_price(40, 50, 0.5, 0.05, 0.281, call=0)

# sigmas = np.empty((3, 3))
sigmas = np.empty((6,))

# for i in range(3):
#   for j in range(3):
#     root = fsolve(func, 0.5)
#     sigmas[i, j] = root

for i in range(6):
  root = fsolve(func, 0.5)
  sigmas[i] = root

sigmas

/tmp/ipython-input-3893136238.py:28: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



array([0.46715642, 0.47791436, 0.47751398, 0.46051389, 0.43225229,
       0.40365966])

## Option pricing and greeks

In [26]:
def euro_option_price(s_0, k, T, r, sigma, div_yield=0, div=0, call=1):
  d1 = (np.log((s_0 - div) / k) + (r - div_yield + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)

  if call:
    return (s_0 * np.exp(-div_yield * T) - div) * norm.cdf(d1) - k * np.exp(-r * T) * norm.cdf(d2)
  else: # put
    return k * np.exp(-r * T) * norm.cdf(-d2) - (s_0 * np.exp(-div_yield * T) - div) * norm.cdf(-d1)

In [27]:
def euro_futures_option_price(f_0, k, T, r, sigma, call=1):
  d1 = (np.log(f_0 / k) + (sigma ** 2) * T / 2) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)

  if call:
    return np.exp(-r * T) * (f_0 * norm.cdf(d1) - k * norm.cdf(d2))
  else:
    return np.exp(-r * T) * (k * norm.cdf(-d2) - f_0 * norm.cdf(-d1))

In [28]:
def variable_volume_dollars(s_0, k, T, r, sigma):

  # This is the expected payoff of a contract that pays off s_T * max(s_T - k, 0)
  # This function is useful to price variable volume products
  # Note that this is just the regular Black Scholes equation with s_0 replaced by s_0 * exp((r + sigma ** 2) * T)

  d1 = (np.log(s_0 / k) + (r + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
  d3 = d1 + sigma * np.sqrt(T)

  return ((s_0 ** 2) * np.exp((r + sigma ** 2) * T) * norm.cdf(d3)) - (k * s_0 * norm.cdf(d1))

In [29]:
def put_call_parity_check(s_0, k, T, r, c, p, div=0):
    if np.isclose(c - p, s_0 - div - k * np.exp(-r * T)):
      return True
    else:
      return False

In [30]:
def prob_option_exercise(s_0, k, T, r, sigma, div_yield=0, div=0, call=1):

  d1 = (np.log((s_0 - div) / k) + (r - div_yield + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)

  if call:
    return norm.cdf(d2)
  else: # put
    return norm.cdf(-d2)

In [31]:
def delta(s_0, k, T, r, sigma, div_yield=0, div=0, call=1):
  d1 = (np.log((s_0 - div) / k) + (r - div_yield + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))

  if call:
    return norm.cdf(d1) * np.exp(-div_yield * T)
  else: # put
    return (norm.cdf(d1) - 1) * np.exp(-div_yield * T)

In [32]:
def theta(s_0, k, T, r, sigma, div_yield=0, div=0, call=1):
  d1 = (np.log((s_0 - div) / k) + (r - div_yield + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)

  if call:
    return (- (s_0 - div) * norm.pdf(d1) * sigma * np.exp(-div_yield * T)) / (2 * np.sqrt(T)) - (r * k * np.exp(-r * T) * norm.cdf(d2)) + (div_yield * s_0 * norm.cdf(d1) * np.exp(-div_yield * T))
  else: # put
    return (- (s_0 - div) * norm.pdf(d1) * sigma * np.exp(-div_yield * T)) / (2 * np.sqrt(T)) + (r * k * np.exp(-r * T) * norm.cdf(-d2)) - (div_yield * s_0 * norm.cdf(-d1) * np.exp(-div_yield * T))

In [33]:
def gamma(s_0, k, T, r, sigma, div_yield=0, div=0):
  d1 = (np.log((s_0 - div) / k) + (r - div_yield + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
  sigma_sqrt_T = sigma * np.sqrt(T)

  return norm.pdf(d1) * np.exp(-div_yield * T) / (s_0 * sigma_sqrt_T)

In [34]:
def vega(s_0, k, T, r, sigma, div_yield=0, div=0, call=1):
  d1 = (np.log((s_0 - div) / k) + (r - div_yield + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))

  return s_0 * np.sqrt(T) * norm.pdf(d1) * np.exp(-div_yield * T)

In [35]:
def rho(s_0, k, T, r, sigma, div_yield=0, div=0, call=1):
  d2 = (np.log((s_0 - div) / k) + (r - div_yield - (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))

  if call:
    return k * T * np.exp(-r * T) * norm.cdf(d2)
  else:
    return -k * T * np.exp(-r * T) * norm.cdf(-d2)

In [36]:
def euro_option_price_lower_bound(s_0, k, T, r, div=0, call=1):

  if call:
    return (s_0 - div) - k * np.exp(-r * T)
  else: # put
    return k * np.exp(-r * T) - (s_0 - div)

In [37]:
euro_option_price(1200, 1200 * (1 - 2.5 / 100), 0.5, 0.06, 0.3, div_yield=0.03, call=0) * 3e5 * 1.5

# (1200 * 3e5 * np.exp(-0.03 * 0.5))

  # - (360 * 0.95) * np.exp(-0.06 * 0.5))

delta(360, 360 * 0.95, 0.5, 0.06, 0.45, div_yield=0.04, call=0) / (np.exp(0.06 * 0.75)) * 3e5 / 250

np.float64(-407.64925952428445)

In [38]:
delta(360, 360 * 0.95, 0.5, 0.06, 0.3, div_yield=0.03, call=0) * 360 / (1200 * 250 * np.exp(0.03 * 0.75)) * 1e6

np.float64(-390.3979094893523)

In [39]:
# # delta check

# (euro_option_price(30 + 1e-5, 30, 1, 0.05, 0.25) - euro_option_price(30, 30, 1, 0.05, 0.25)) / 1e-5

# # gamma check

# (delta(30 + 1e-5, 30, 1, 0.05, 0.25) - delta(30, 30, 1, 0.05, 0.25)) / 1e-5

# # vega check

# (euro_option_price(30, 30, 1, 0.05, 0.25 + 1e-5) - euro_option_price(30, 30, 1, 0.05, 0.25)) / 1e-5

# # theta check

# (euro_option_price(30, 30, 1 - 1e-5, 0.05, 0.25) - euro_option_price(30, 30, 1, 0.05, 0.25)) / 1e-5

# # rho check

# (euro_option_price(30, 30, 1, 0.05 + 1e-5, 0.25) - euro_option_price(30, 30, 1, 0.05, 0.25)) / 1e-5

euro_option_price(49, 50, 20 / 52, 0.05, 0.20)

np.float64(2.4005273232717137)

In [40]:
delta(49, 50, 20 / 52, 0.05, 0.20)

np.float64(0.5216046610663964)

In [41]:
gamma(49, 50, 20 / 52, 0.05, 0.20)

np.float64(0.06554403934784439)

In [42]:
vega(49, 50, 20 / 52, 0.05, 0.20)

np.float64(12.105479882628801)

In [43]:
theta(49, 50, 20 / 52, 0.05, 0.20) + (0.05 * 49 * delta(49, 50, 20 / 52, 0.05, 0.20)) + (0.5 * (0.2 ** 2) * (49 ** 2) * gamma(49, 50, 20 / 52, 0.05, 0.20)) - (0.05 * euro_option_price(49, 50, 20 / 52, 0.05, 0.2))

np.float64(1.0547118733938987e-15)

In [44]:
rho(49, 50, 20 / 52, 0.05, 0.20)

np.float64(8.906961949608348)

## Sensitivity of option prices to variables

In [45]:
variable_so = np.linspace(0.001, 100, 100)
variable_k = np.linspace(0.001, 100, 100)
variable_T = np.linspace(0.001, 2, 100)
variable_sigma = np.linspace(0.001, 0.5, 100)
variable_r = np.linspace(0.001, 0.08, 100)

In [46]:
fig = make_subplots(rows=5, cols=2, subplot_titles=("Call", "Put"))

fig.add_trace(go.Scatter(
    x=variable_so,
    y=euro_option_price(variable_so, 50, 1 , 0.05, 0.3),
    mode="lines",
    name="Call price"
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=variable_so,
    y=euro_option_price(variable_so, 50, 1 , 0.05, 0.3, call=0),
    mode="lines",
    name="Put price"
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=variable_k,
    y=euro_option_price(50, variable_k, 1 , 0.05, 0.3),
    mode="lines",
    name="Call price"
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=variable_k,
    y=euro_option_price(50, variable_k, 1 , 0.05, 0.3, call=0),
    mode="lines",
    name="Put price"
), row=2, col=2)

fig.add_trace(go.Scatter(
    x=variable_T,
    y=euro_option_price(50, 50, variable_T, 0.05, 0.3),
    mode="lines",
    name="Call price"
), row=3, col=1)

fig.add_trace(go.Scatter(
    x=variable_T,
    y=euro_option_price(50, 50, variable_T, 0.05, 0.3, call=0),
    mode="lines",
    name="Put price"
), row=3, col=2)

fig.add_trace(go.Scatter(
    x=variable_r,
    y=euro_option_price(50, 50, 1, variable_r, 0.3),
    mode="lines",
    name="Call price"
), row=4, col=1)

fig.add_trace(go.Scatter(
    x=variable_r,
    y=euro_option_price(50, 50, 1, variable_r, 0.3, call=0),
    mode="lines",
    name="Put price"
), row=4, col=2)

fig.add_trace(go.Scatter(
    x=variable_sigma,
    y=euro_option_price(50, 50, 1, 0.05, variable_sigma),
    mode="lines",
    name="Call price"
), row=5, col=1)

fig.add_trace(go.Scatter(
    x=variable_sigma,
    y=euro_option_price(50, 50, 1, 0.05, variable_sigma, call=0),
    mode="lines",
    name="Put price"
), row=5, col=2)

fig.update_xaxes(title_text='Underlying price', row=1, col=1)
fig.update_yaxes(title_text='Option price', row=1, col=1)
fig.update_xaxes(title_text='Strike price', row=2, col=1)
fig.update_yaxes(title_text='Option price', row=2, col=1)
fig.update_xaxes(title_text='Time to maturity', row=3, col=1)
fig.update_yaxes(title_text='Option price', row=3, col=1)
fig.update_xaxes(title_text='Risk-free rate', row=4, col=1)
fig.update_yaxes(title_text='Option price', row=4, col=1)
fig.update_xaxes(title_text='Volatility', row=5, col=1)
fig.update_yaxes(title_text='Option price', row=5, col=1)

fig.update_xaxes(title_text='Underlying price', row=1, col=2)
fig.update_yaxes(title_text='Option price', row=1, col=2)
fig.update_xaxes(title_text='Strike price', row=2, col=2)
fig.update_yaxes(title_text='Option price', row=2, col=2)
fig.update_xaxes(title_text='Time to maturity', row=3, col=2)
fig.update_yaxes(title_text='Option price', row=3, col=2)
fig.update_xaxes(title_text='Risk-free rate', row=4, col=2)
fig.update_yaxes(title_text='Option price', row=4, col=2)
fig.update_xaxes(title_text='Volatility', row=5, col=2)
fig.update_yaxes(title_text='Option price', row=5, col=2)

fig.update_layout(height=1000, width=1600, title_text="Option price sensitivity")
fig.update_layout(showlegend=False)

fig.show()

## Payoffs of various option trading strategies

In [47]:
# Note that this analysis ignores the impact of discounting for simplicity

# Defining variables for generating graphs

variable_s_t = np.linspace(0.001, 100, 100) # Underlying price at time of observation

s_0 = 32
k_1 = 25
k_2 = 30 # for butterfly/diagonal spreads and strangles
k_3 = 35 # for butterfly/diagonal spreads
T_1 = 1
T_2 = 1.5 # for calendar/diagonal spreads
# T_3 =
r = 0.05
sigma = 0.30

### Call and put option

In [48]:
def euro_call_option_payoff(s_T, k, T, r, sigma, call=1): # This is the payoff from buying. The payoff from selling would be the negative of this.
  if call:
    return np.maximum(np.exp(-r * T) * (s_T - k), 0)
  else: # put
    return np.maximum(-np.exp(-r * T) * (s_T - k), 0)

In [49]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=variable_s_t, y=euro_call_option_payoff(variable_s_t, k_1, T_1, r, sigma, call=1) - euro_option_price(s_0, k_1, r, T_1, sigma, call=1), mode='lines', name='Call option payoff'))
fig.add_trace(go.Scatter(x=variable_s_t, y=euro_call_option_payoff(variable_s_t, k_1, T_1, r, sigma, call=0) - euro_option_price(s_0, k_1, r, T_1, sigma, call=0), mode='lines', name='Put option payoff'))

fig.update_layout(
    title="Option payoff at expiry",
    xaxis_title="Underlying price",
    yaxis_title="Payoff"
)

fig.show()

### Bull spread

In [50]:
fig = go.Figure()

# fig.add_trace(go.Scatter(x=variable_s_t, y=euro_call_option_payoff(variable_s_t, k_1, T_1, r, sigma, call=1) -
#                          euro_call_option_payoff(variable_s_t, k_2, T_1, r, sigma, call=1) -
#                          euro_option_price(s_0, k_1, T_1, r, sigma, call=1) +
#                          euro_option_price(s_0, k_2, T_1, r, sigma, call=1)
#                          ,
#                          mode='lines')) # using calls

fig.add_trace(go.Scatter(x=variable_s_t, y=euro_call_option_payoff(variable_s_t, k_1, T_1, r, sigma, call=0) -
                         euro_call_option_payoff(variable_s_t, k_2, T_1, r, sigma, call=0) -
                         euro_option_price(s_0, k_1, T_1, r, sigma, call=0) +
                         euro_option_price(s_0, k_2, T_1, r, sigma, call=0)
                         ,
                         mode='lines')) # using puts

fig.update_layout(
    title="Bull spread payoff",
    xaxis_title="Underlying price",
    yaxis_title="Payoff"
)

fig.show()

### Bear spread

In [51]:
fig = go.Figure()

# fig.add_trace(go.Scatter(x=variable_s_t, y=euro_call_option_payoff(variable_s_t, k_1, T_1, r, sigma, call=1) -
#                          euro_call_option_payoff(variable_s_t, k_2, T_1, r, sigma, call=1) -
#                          euro_option_price(s_0, k_1, T_1, r, sigma, call=1) +
#                          euro_option_price(s_0, k_2, T_1, r, sigma, call=1)
#                          ,
#                          mode='lines')) # using calls

fig.add_trace(go.Scatter(x=variable_s_t, y=-euro_call_option_payoff(variable_s_t, k_1, T_1, r, sigma, call=0) +
                         euro_call_option_payoff(variable_s_t, k_2, T_1, r, sigma, call=0) +
                         euro_option_price(s_0, k_1, T_1, r, sigma, call=0) -
                         euro_option_price(s_0, k_2, T_1, r, sigma, call=0)
                         ,
                         mode='lines')) # using puts

fig.update_layout(
    title="Bear spread payoff",
    xaxis_title="Underlying price",
    yaxis_title="Payoff"
)

fig.show()

### Butterfly spread

In [52]:
fig = go.Figure()

# fig.add_trace(go.Scatter(x=variable_s_t, y=euro_call_option_payoff(variable_s_t, k_1, T_1, r, sigma, call=1) +
#                          euro_call_option_payoff(variable_s_t, k_3, T_1, r, sigma, call=1) -
#                          2 * euro_call_option_payoff(variable_s_t, k_2, T_1, r, sigma, call=1) -
#                          euro_option_price(s_0, k_1, T_1, r, sigma, call=1) -
#                          euro_option_price(s_0, k_3, T_1, r, sigma, call=1) +
#                          2 * euro_option_price(s_0, k_2, T_1, r, sigma, call=1)
#                          ,
#                          mode='lines')) # using calls

fig.add_trace(go.Scatter(x=variable_s_t, y=euro_call_option_payoff(variable_s_t, k_1, T_1, r, sigma, call=0) +
                         euro_call_option_payoff(variable_s_t, k_3, T_1, r, sigma, call=0) -
                         2 * euro_call_option_payoff(variable_s_t, k_2, T_1, r, sigma, call=0) -
                         euro_option_price(s_0, k_1, T_1, r, sigma, call=0) -
                         euro_option_price(s_0, k_3, T_1, r, sigma, call=0) +
                         2 * euro_option_price(s_0, k_2, T_1, r, sigma, call=0)
                         ,
                         mode='lines')) # using puts

fig.update_layout(
    title="Butterfly spread payoff",
    xaxis_title="Underlying price",
    yaxis_title="Payoff"
)

fig.show()

### Calendar spread

In [53]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=variable_s_t, y=euro_option_price(variable_s_t, k_1, T_2, r, sigma, call=1) - # selling longer-term option
                         euro_call_option_payoff(variable_s_t, k_1, T_1, r, sigma, call=1) + # payoff at expiry of shorter-term option
                         euro_option_price(s_0, k_1, T_1, r, sigma, call=1) - # cost of shorter-term option
                         euro_option_price(s_0, k_1, T_2, r, sigma, call=1), # buying longer-term option
                         mode='lines')) # using calls

fig.update_layout(
    title="Calendar spread payoff",
    xaxis_title="Underlying price",
    yaxis_title="Payoff"
)

fig.show() # Payoff like a butterfly spread

### Diagonal spread

In [54]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=variable_s_t, y=euro_option_price(variable_s_t, k_2, T_2, r, sigma, call=1) - # selling longer-term option
                         euro_call_option_payoff(variable_s_t, k_1, T_1, r, sigma, call=1) + # payoff at expiry of shorter-term option
                         euro_option_price(s_0, k_1, T_1, r, sigma, call=1) - # cost of shorter-term option
                         euro_option_price(s_0, k_2, T_2, r, sigma, call=1), # buying longer-term option
                         mode='lines'))

fig.update_layout(
    title="Diagonal spread payoff",
    xaxis_title="Underlying price",
    yaxis_title="Payoff"
)

fig.show() # Payoff like a bull/bear spread depending on whether k1 < k2 or k1 > k2

### Straddle

In [55]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=variable_s_t, y=euro_call_option_payoff(variable_s_t, k_1, T_1, r, sigma, call=1) -
                         euro_option_price(s_0, k_1, r, T_1, sigma, call=1) +
                         euro_call_option_payoff(variable_s_t, k_1, T_1, r, sigma, call=0) -
                         euro_option_price(s_0, k_1, r, T_1, sigma, call=0)
                         , mode='lines'))


fig.update_layout(
    title="Straddle payoff at expiry",
    xaxis_title="Underlying price",
    yaxis_title="Payoff"
)

fig.show()

### Strangle

In [56]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=variable_s_t, y=euro_call_option_payoff(variable_s_t, k_1, T_1, r, sigma, call=1) -
                         euro_option_price(s_0, k_1, r, T_1, sigma, call=1) +
                         euro_call_option_payoff(variable_s_t, k_2, T_1, r, sigma, call=0) -
                         euro_option_price(s_0, k_2, r, T_1, sigma, call=0)
                         , mode='lines'))


fig.update_layout(
    title="Strangle payoff at expiry",
    xaxis_title="Underlying price",
    yaxis_title="Payoff"
)

fig.show()

## Greeks visualized

### Delta - Slope of option price vs underlying price

In [57]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Call", "Put"))

fig.add_trace(go.Scatter(
    x=variable_so,
    y=delta(variable_so, 50, 1 , 0.05, 0.3),
    mode="lines",
    name="Call delta"
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=variable_so,
    y=delta(variable_so, 50, 1 , 0.05, 0.3, call=0),
    mode="lines",
    name="Put delta"
), row=1, col=2)

fig.update_xaxes(title_text='Underlying price', row=1, col=1)
fig.update_yaxes(title_text='Option delta', row=1, col=1)

fig.update_xaxes(title_text='Underlying price', row=1, col=2)
fig.update_yaxes(title_text='Option delta', row=1, col=2)

### Theta - Slope of option price vs time

In [58]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Call", "Put"))

fig.add_trace(go.Scatter(
    x=variable_so,
    y=theta(variable_so, 50, 1, 0.05, 0.3),
    mode="lines",
    name="Call theta"
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=variable_so,
    y=theta(variable_so, 50, 1, 0.05, 0.3, call=0),
    mode="lines",
    name="Put theta"
), row=1, col=2)

fig.update_xaxes(title_text='Underlying price', row=1, col=1)
fig.update_yaxes(title_text='Option theta', row=1, col=1)

fig.update_xaxes(title_text='Underlying price', row=1, col=2)
fig.update_yaxes(title_text='Option theta', row=1, col=2)

In [59]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Call", "Put"))

fig.add_trace(go.Scatter(
    x=variable_T,
    y=theta(100, 50, variable_T , 0.05, 0.3),
    mode="lines",
    name="Call theta"
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=variable_T,
    y=theta(100, 50, variable_T, 0.05, 0.3, call=0),
    mode="lines",
    name="Put theta"
), row=1, col=2)

fig.update_xaxes(title_text='Time to maturity', row=1, col=1)
fig.update_yaxes(title_text='Option theta', row=1, col=1)

fig.update_xaxes(title_text='Time to maturity', row=1, col=2)
fig.update_yaxes(title_text='Option theta', row=1, col=2)

### Gamma - Slope of option delta vs underlying price

In [60]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=variable_so,
    y=gamma(variable_so, 50, 1, 0.05, 0.3),
    mode="lines",
    name="Option gamma"
))

fig.update_layout(
    title="Option gamma",
    xaxis_title="Underlying price",
    yaxis_title="Option gamma"
)

fig.show()

In [61]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=variable_T,
        y=gamma(100, 50, variable_T , 0.05, 0.3),
        mode="lines",
        name="Option gamma"
    )
)

fig.update_layout(
    title="Option gamma",
    xaxis_title="Time to maturity",
    yaxis_title="Option gamma"
)

fig.show()

### Vega - Slope of option price vs volatility

In [62]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=variable_so,
    y=vega(variable_so, 50, 1, 0.05, 0.3),
    mode="lines",
    name="Option vega"
))

fig.update_layout(
    title="Option vega",
    xaxis_title="Underlying price",
    yaxis_title="Option vega"
)

fig.show()

### Option Rho

In [63]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=variable_so,
    y=rho(variable_so, 50, 1, 0.05, 0.3),
    mode="lines",
    name="Option rho"
))

fig.update_layout(
    title="Option rho",
    xaxis_title="Underlying price",
    yaxis_title="Option rho"
)

fig.show()

## Geometric Brownian Motion

In [64]:
S_0 = 49 # Initial price
# L_0 = 49 # Initial load/volume/notional
mu_S = 0.05 # Price drift
sigma_S = 0.2 # Price volatility
T = 20 / 52 # Time to maturity in years
total_steps = 1000
time_step = T / total_steps # Dividing the time to maturity into discrete steps)

num_simulations = 10000 # number of simulations

Prices = np.zeros((num_simulations, total_steps))

for t in range(total_steps):
  if t == 0:
    Prices[:, t] = S_0
  else:
    drift_term = mu_S * Prices[:, t - 1] * (time_step)
    diffusion_term = sigma_S * Prices[:, t - 1] * np.sqrt(time_step) * np.random.standard_normal(size=num_simulations)
    Prices[:, t] = Prices[:, t - 1] + drift_term + diffusion_term

fig = go.Figure()

# Plotting 5 of the {num_simulations} paths
for i in range(5):
  fig.add_trace(
      go.Scatter(
          x=np.linspace(0, T, total_steps),
          y=Prices[i, :],
          name=f'Price path {i + 1}'
  )
      )

fig.update_layout(
    title='Geometric Brownian Motion',
    xaxis_title = 'Time in years',
    yaxis_title = 'Prices'
)

In [65]:
# Verifying that the simulation is correct by testing it on a call option
k = 50

print(f'Theoretical value of European call option {euro_option_price(S_0, k, T, mu_S, sigma_S)}')
print(f'Empirical value of European call option {(np.maximum(Prices[:, total_steps - 1] - k, 0).mean()) * np.exp(- mu_S * T)}')

Theoretical value of European call option 2.4005273232717137
Empirical value of European call option 2.4306819310479413


### Dynamic Delta Hedging

In [66]:
# These are the dynamic hedge results from shorting a call option and hedging it by going long on the underlying. The portfolio consists of the option and dynamically changing units of the underlying.

dynamic_hedge_results_sample = pd.DataFrame(Prices[np.random.randint(0, num_simulations), :])

dynamic_hedge_results_sample.columns = ['Underlying_price']

dynamic_hedge_results_sample = dynamic_hedge_results_sample.assign(
  t=lambda DF: np.linspace(0, T, total_steps), # in years
  time_step=lambda DF: DF.t.diff().bfill(), # in years
  Time_to_expiry=lambda DF: T - DF.t, # in years
  Option_price=lambda DF: DF.apply(lambda DF: euro_option_price(DF.Underlying_price, k, DF.Time_to_expiry, mu_S, sigma_S), axis=1),
  Delta=lambda DF: -DF.apply(lambda DF: delta(DF.Underlying_price, k, DF.Time_to_expiry, mu_S, sigma_S), axis=1),
  Theta=lambda DF: -DF.apply(lambda DF: theta(DF.Underlying_price, k, DF.Time_to_expiry, mu_S, sigma_S), axis=1),
  Gamma=lambda DF: -DF.apply(lambda DF: gamma(DF.Underlying_price, k, DF.Time_to_expiry, mu_S, sigma_S), axis=1),
  Change_in_Delta=lambda DF: DF.Delta.diff(),
  Underlying_purchased=lambda DF: -np.where(DF.index==0, DF.Delta, DF.Change_in_Delta),
  Underlying_portfolio=lambda DF: DF.Underlying_purchased.cumsum(), # Number of units of underlying in portfolio
  Cost_of_underlying_purchased=lambda DF: DF.Underlying_purchased * DF.Underlying_price,
  Cumulative_cost_with_interest=np.nan, # placeholder
  Interest_cost=np.nan, # placeholder
  Portfolio_change=lambda DF: (-DF.Option_price.diff() + DF.Underlying_portfolio * DF.Underlying_price.diff()), # Short option, long underlying
  Greek_check=lambda DF: DF.Portfolio_change.shift(-1) - (DF.Theta * DF.time_step) - 0.5 * DF.Gamma * (DF.Underlying_price.diff(-1) ** 2) # Checking if equation 14.6 in book is satisfied
)

# Updating columns 'Cumulative_cost_with_interest' and 'Interest_cost' using a loop below since I wasn't able to use a vectorized version above
for i in range(total_steps):

  if i == 0:
    dynamic_hedge_results_sample.loc[i, 'Cumulative_cost_with_interest'] = dynamic_hedge_results_sample.loc[i, 'Cost_of_underlying_purchased']

  else:
    dynamic_hedge_results_sample.loc[i, 'Cumulative_cost_with_interest'] = dynamic_hedge_results_sample.loc[i - 1, 'Cumulative_cost_with_interest'] + dynamic_hedge_results_sample.loc[i - 1, 'Interest_cost'] + dynamic_hedge_results_sample.loc[i, 'Cost_of_underlying_purchased']

  dynamic_hedge_results_sample.loc[i, 'Interest_cost'] = dynamic_hedge_results_sample.loc[i, 'Cumulative_cost_with_interest'] * mu_S * dynamic_hedge_results_sample.loc[i, 'time_step']

(
dynamic_hedge_results_sample
# dynamic_hedge_results_sample.Greek_check.describe()
# .pipe(
#     lambda DF: DF.Cumulative_cost_with_interest[total_steps - 1] - k * (DF.Underlying_price[total_steps - 1] > k)
#     # Receive strike price if option closes in the money with delta of 1, else 0. This number should be approximately equal to the value of the option
# )
.pipe(
    px.line,
    x=dynamic_hedge_results_sample.index,
    # y='Cumulative_cost_with_interest',
    y=-dynamic_hedge_results_sample.Theta,
    title=f'Dynamic delta hedging results for one of the {num_simulations} price paths',
    labels={
        'x': 'Time_step',
    }
)
)


/tmp/ipython-input-3400423573.py:2: RuntimeWarning:

divide by zero encountered in scalar divide

/tmp/ipython-input-2043817593.py:2: RuntimeWarning:

divide by zero encountered in scalar divide

/tmp/ipython-input-797921792.py:2: RuntimeWarning:

divide by zero encountered in scalar divide

/tmp/ipython-input-797921792.py:6: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipython-input-332326176.py:2: RuntimeWarning:

divide by zero encountered in scalar divide

/tmp/ipython-input-332326176.py:5: RuntimeWarning:

invalid value encountered in scalar divide



## Variable volume swap

### Simple variable volume swap

Now, we will price a variable volume forward or swap (similar to the Full Requirement deals) using a simplified model of volume (N) being a constant (equal to 1) times the price.

The payoff of this contract is S_T * (S_T - k).

In [67]:
def variable_volume_swap_simple_strike(S_0, r, T, sigma_S):

  strike = S_0 * np.exp((r + (sigma_S ** 2)) * T)

  return strike

In [68]:
def variable_volume_swap_simple_expected_payoff_analytical(S_0, r, T, sigma_S, K):

  return (S_0 ** 2) * np.exp((r + (sigma_S ** 2)) * T) - K * S_0

In [69]:
def variable_volume_swap_simple_delta(S_0, r, T, sigma_S, K):

  delta = 2 * S_0 * np.exp((r + (sigma_S ** 2)) * T) - K # This is obtained by differentiating the analytical payoff with respect to S_0

  return delta

In [70]:
def variable_volume_swap_simple_expected_payoff_empirical(strike):

  return ((Prices[:, total_steps - 1] - strike) * Prices[:, total_steps - 1]).mean()

In [71]:
analytical_strike = variable_volume_swap_simple_strike(S_0, 0, T, sigma_S)
analytical_delta = variable_volume_swap_simple_delta(S_0, 0, T, sigma_S, analytical_strike)

print(f'Analytical strike and delta of variable volume swap {analytical_strike:.2f}, {analytical_delta:.2f}')
print(f'Empirical strike of variable volume swap {fsolve(variable_volume_swap_simple_expected_payoff_empirical, 10)}')

Analytical strike and delta of variable volume swap 49.76, 49.76
Empirical strike of variable volume swap [50.80779743]


In [72]:
bump_spot = 1e-10

bump_payoff = variable_volume_swap_simple_expected_payoff_analytical(S_0 + bump_spot, 0, T, sigma_S, analytical_strike)
- variable_volume_swap_simple_expected_payoff_analytical(S_0, 0, T, sigma_S, analytical_strike)

print(f'Empirical delta of variable volume swap {bump_payoff / bump_spot}')

Empirical delta of variable volume swap 49.76300260750577


In [73]:
variable_volume_swap_simple_expected_payoff_analytical(S_0, 0, T, sigma_S, 200)

np.float64(-7361.775933689514)

In [74]:
variable_volume_swap_simple_expected_payoff_empirical(200)

np.float64(-7464.140590374373)

### Complex variable volume swap



Now, let's model the volume in a slightly more sophisticated way. We know that in electricity markets, when prices increase, the volume tends to increase, but the effect tends to diminish at very high or low prices. This allows us to model the volume similar to a bull spread (see Arvind's document on variable volume forwards for more details).

In [75]:
def variable_volume_swap_strike(N_L, N_H, K_L, K_H, S_0, sigma_S, T):

  """
  This function models prices as a function of volume as a call spread and calculates the no-arbitrage strike and delta of a variable-volume swap

  """

  lev = (N_H - N_L) / (K_H - K_L)

  S_tilde = S_0 * np.exp((sigma_S ** 2) * T) # This is the modified version based on Girsanov's theorem

  # BS - Black Scholes price
  # CS - Call spread

  BS_L = euro_option_price(S_0, K_L, T, 0, sigma_S)
  BS_H = euro_option_price(S_0, K_H, T, 0, sigma_S)

  CS = BS_L - BS_H

  BS_S_tilde_L = euro_option_price(S_tilde, K_L, T, 0, sigma_S)
  BS_S_tilde_H = euro_option_price(S_tilde, K_H, T, 0, sigma_S)

  CS_tilde = BS_S_tilde_L - BS_S_tilde_H

  variable_volume_dollars_low = variable_volume_dollars(S_0, K_L, T, 0, sigma_S)
  variable_volume_dollars_high = variable_volume_dollars(S_0, K_H, T, 0, sigma_S)

  variable_volume_dollars_spread = variable_volume_dollars_low - variable_volume_dollars_high

  variable_volume_strike = (N_L * S_0 + lev * S_0 * CS_tilde) / (N_L + lev * CS)

  # delta_CS = delta(S_0, K_L, T, 0, sigma_S) - delta(S_tilde, K_H, T, 0, sigma_S)

  # delta_CS_tilde = (delta(S_tilde, K_L, T, 0, sigma_S) - delta(S_tilde, K_H, T, 0, sigma_S)) * np.exp((sigma_S ** 2) * T)

  # delta_variable_volume_swap = N_L + lev * (CS_tilde + delta_CS_tilde * S_0 - variable_volume_strike * delta_CS)

  # print(f'Theoretical strike of variable volume swap using first principles {(N_L * S_0 + lev * variable_volume_dollars_spread) / (N_L + lev * CS)}') # This serves as a check

  return variable_volume_strike


In [76]:
def variable_volume_swap_delta(N_L, N_H, K_L, K_H, S_0, sigma_S, T, K):

  lev = (N_H - N_L) / (K_H - K_L)

  S_tilde = S_0 * np.exp((sigma_S ** 2) * T) # This is the modified version based on Girsanov's theorem

  # BS - Black Scholes price
  # CS - Call spread

  BS_L = euro_option_price(S_0, K_L, T, 0, sigma_S)
  BS_H = euro_option_price(S_0, K_H, T, 0, sigma_S)

  CS = BS_L - BS_H

  BS_S_tilde_L = euro_option_price(S_tilde, K_L, T, 0, sigma_S)
  BS_S_tilde_H = euro_option_price(S_tilde, K_H, T, 0, sigma_S)

  CS_tilde = BS_S_tilde_L - BS_S_tilde_H

  # variable_volume_dollars_low = variable_volume_dollars(S_0, K_L, T, 0, sigma_S)
  # variable_volume_dollars_high = variable_volume_dollars(S_0, K_H, T, 0, sigma_S)

  # variable_volume_dollars_spread = variable_volume_dollars_low - variable_volume_dollars_high

  # variable_volume_strike = (N_L * S_0 + lev * S_0 * CS_tilde) / (N_L + lev * CS)

  delta_CS = delta(S_0, K_L, T, 0, sigma_S) - delta(S_0, K_H, T, 0, sigma_S)

  delta_CS_tilde = (delta(S_tilde, K_L, T, 0, sigma_S) - delta(S_tilde, K_H, T, 0, sigma_S)) * np.exp((sigma_S ** 2) * T)

  delta_variable_volume_swap = N_L + lev * (CS_tilde + delta_CS_tilde * S_0 - K * delta_CS)

  # print('lev', lev)
  # print('CS', CS)
  # print('CS_tilde', CS_tilde)
  # print('delta_CS', delta_CS)
  # print('delta_CS_tilde', delta_CS_tilde)

  return delta_variable_volume_swap

In [77]:
def variable_volume_swap_expected_payoff_analytical(N_L, N_H, K_L, K_H, S_0, sigma_S, T, K):

  """
  Returns the expected payoff of a variable volume swap. Created for the purpose of empirically calculating the delta of the swap
  """

  lev = (N_H - N_L) / (K_H - K_L)

  # BS - Black Scholes price
  # CS - Call spread

  BS_L = euro_option_price(S_0, K_L, T, 0, sigma_S)
  BS_H = euro_option_price(S_0, K_H, T, 0, sigma_S)

  CS = BS_L - BS_H

  variable_volume_dollars_low = variable_volume_dollars(S_0, K_L, T, 0, sigma_S)
  variable_volume_dollars_high = variable_volume_dollars(S_0, K_H, T, 0, sigma_S)

  variable_volume_dollars_spread = variable_volume_dollars_low - variable_volume_dollars_high

  return (N_L * S_0 + lev * variable_volume_dollars_spread) - K * (N_L + lev * CS)

In [78]:
N_L = 53.43
N_H = 191.23
K_L = 20
K_H = 900

lev = (N_H - N_L) / (K_H - K_L)

analytical_strike = variable_volume_swap_strike(N_L, N_H, K_L, K_H, S_0, sigma_S, T)
analytical_delta = variable_volume_swap_delta(N_L, N_H, K_L, K_H, S_0, sigma_S, T, analytical_strike)

print(f'Analytical strike and delta of variable volume swap {analytical_strike:.2f}, {analytical_delta:.2f}')

Analytical strike and delta of variable volume swap 49.10, 58.19


In [79]:
# euro_option_price((S_0 * np.exp(sigma_S ** 2) * T), K_H, T, 0, sigma_S)

# euro_option_price(S_0, K_H, T, 0, sigma_S)

Now, we'll empirically calculate the no-arbitrage strike of the variable-volume swap, which would be the price we quote to the customer

In [80]:
def variable_volume_swap_expected_payoff_empirical(strike):
  Volume = N_L + lev * (np.maximum(Prices[:, total_steps - 1] - K_L, 0) - np.maximum(Prices[:, total_steps - 1] - K_H, 0))
  return ((Prices[:, total_steps - 1] - strike) * Volume).mean()

In [81]:
print(f'Empirical strike of variable volume swap {fsolve(variable_volume_swap_expected_payoff_empirical, 10)}')

Empirical strike of variable volume swap [50.1351381]


In [82]:
# Checking expected payoff

variable_volume_swap_expected_payoff_empirical(100), variable_volume_swap_expected_payoff_analytical(N_L, N_H, K_L, K_H, S_0, sigma_S, T, 100)

(np.float64(-2898.7682070805045), np.float64(-2950.6990041618365))

Now, we'll empirically calculate  the delta of the variable-volume swap (which would be the volume to dynamically hedge the deal) and compare with the analytical results above

In [83]:
bump_spot = 1e-10

# strike = variable_volume_swap_strike_and_delta(N_L, N_H, K_L, K_H, S_0, sigma_S, T)[0]

bump_payoff = variable_volume_swap_expected_payoff_analytical(N_L, N_H, K_L, K_H, S_0 + bump_spot, sigma_S, T, analytical_strike)
- variable_volume_swap_expected_payoff_analytical(N_L, N_H, K_L, K_H, S_0, sigma_S, T, analytical_strike)

print(f'Empirical delta {bump_payoff / bump_spot}')

Empirical delta 58.19401849294081


In [84]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=np.linspace(0, 400),
        y=N_L + lev * (np.maximum(np.linspace(0, 400) - K_L, 0) - np.maximum(np.linspace(0, 400) - K_H, 0))
    )
)

fig.update_layout(
    title='Variable volume model',
    xaxis_title='Underlying price',
    yaxis_title='Volume'
)

## Value at Risk (VaR)

This is based on the quadratic model

In [85]:
delta = 12 # of the portfolio
gamma = -2.6 # of the portfoliio
S = 10 # price of underlying
daily_vol = 0.02

# Moments of the distribution of the change in the portfolio - see page 358 in Options, Futures and Other Derivatives for derivation
first_moment = 0.5 * S**2 * gamma * daily_vol**2
second_moment = S**2 * delta**2 * daily_vol**2 + 0.75 * S**4 * gamma**2 * daily_vol**4
third_moment = 4.5 * S**4 * delta**2 * gamma * daily_vol**4 + (15 / 8) * S**6 * gamma**3 * daily_vol**6

first_moment, second_moment, third_moment

(-0.052000000000000005, 5.768112, -2.69778912)

In [86]:
variance = second_moment - first_moment**2
std_dev = np.sqrt(variance)
skewness = (third_moment - 3 * second_moment * first_moment + 2 * first_moment**3) / std_dev**3

# Using the Corning-Fisher expansion to estimate the VaR using three moments

z_q = norm.ppf(0.05) # P-1 of standard normal
w_q = z_q + (1 / 6) * (z_q**2 - 1) * skewness

print(first_moment + w_q * std_dev) # 1-day 99% VaR using just all three moments based on Corning Fisher
print(norm.ppf(0.05, loc=first_moment, scale=std_dev)) # 1-day 99% VaR using just first two moments

-4.090162001160956
-4.001501471653016


## Binomial Tree

In [348]:
def american_option_price(S_0: float, k: float, T: float, r: float, sigma: float, N: int, div_yield: float = 0, div: float = 0, T_div: float = 0, call: bool = 0, american: bool = 1):

    """ Constructs binomial tree for American and European call and put options and returns a tuple consisting of the option price, delta, gamma, theta and the tree itself

    Args:
        S_0: The initial price of the underlying
        k: Strike price
        T: Time to maturity in years
        r: Continuously compundeded interest rate in percent per annum
        sigma: Volatility of the underlying
        N: Number of steps in the tree
        div_yield: Continuously compounded dividend yield. Set to 0 if no dividend yield.
        div: Present value of dividends. Set to 0 if no dividends.
        T_div: Time to dividend (Ex-dividend time). Set to 0 if no dividends.
        call: 0 for put, 1 for call
        euro: 0 for European options, 1 for American options

    Returns: Tuple consisting of the following
        option_price: Option price
        delta: Delta of the option
        gamma: Gamma of the option
        theta: Theta of the option
        binomial_tree: numpy.ndarray - (time ID, level ID, first level price and second level option value)

  """

    delta_T = T / N
    i_div = np.floor(T_div / delta_T) # Find the step of the tree right before the ex-dividend date
    fv_div = div * np.exp(r * T_div) # Future value of dividend (the actual dollar figure given)

    u = np.exp(sigma * np.sqrt(delta_T))
    d = np.exp(-sigma * np.sqrt(delta_T))
    a = np.exp((r - div_yield) * delta_T)
    p = (a - d) / (u - d)

    binary = -1 if call else 1 # for toggling between call and put

    binomial_tree = np.zeros((N + 1, N + 1, 2)) # In 3rd dimension, first level is stock price and second level is option value

    # Populating the tree with the stock prices first
    for i in range(N + 1): # i iterates through the varies steps
      for j in range(i + 1): # j iterates through the branches at each step
        stock_component = (S_0 - div) * u**j * d**(i - j)
        dividend_component = fv_div * np.exp(-r * (T_div - i * delta_T)) * (i <= i_div) # Adding the present value of dividends until right before the ex-dividend date
        binomial_tree[i, j, 0] = stock_component + dividend_component

    # Now calculating the option prices - working backward from end of tree
    binomial_tree[N, :, 1] = np.maximum((k - binomial_tree[N, :, 0]) * binary, 0) # Evaluating the value of the option at expiry

    for i in reversed(range(N)): # Going backwards to calculate option value at each step hence reversed
      for j in range(i + 1):
        binomial_tree[i, j, 1] = np.maximum(
            (k - binomial_tree[i, j, 0]) * binary * american, # allowing for early exercise of put (if American). If European, we force it 0 - which is another way of saying - no early exercise
            (p * binomial_tree[i + 1, j + 1, 1] + (1 - p) * binomial_tree[i + 1, j, 1]) * np.exp(-r * delta_T)
      )

    # Calculating the option price and greeks at time 0
    option_price = binomial_tree[0, 0, 1]

    delta = (binomial_tree[1, 0, 1] - binomial_tree[1, 1, 1]) / (binomial_tree[1, 0, 0] - binomial_tree[1, 1, 0])

    # Calculating the different deltas after the first step in order to calculate gamma
    delta_1 = (binomial_tree[2, 2, 1] - binomial_tree[2, 1, 1]) / (binomial_tree[2, 2, 0] - binomial_tree[2, 1, 0])
    delta_2 = (binomial_tree[2, 1, 1] - binomial_tree[2, 0, 1]) / (binomial_tree[2, 1, 0] - binomial_tree[2, 0, 0])
    h = 0.5 * (binomial_tree[2, 2, 0] - binomial_tree[2, 0, 0])
    gamma = (delta_1 - delta_2) / (h)

    theta = (binomial_tree[2, 1, 1] - binomial_tree[0, 0, 1]) / (2 * delta_T)

    return option_price, delta, gamma, theta, binomial_tree # returning the price, greeks and the tree itself


In [350]:
S_0 = 484
k = 480
T = 2 / 12
r = 0.1
sigma = 0.25
div_yield = 0.03
T_div = 0
div = 0 # 2 * np.exp(-r * T_div)

Ns = np.arange(2, 101, 1)
american_option_prices = np.zeros(len(Ns))

for N in Ns:
  american_option_prices[N - 2] = american_option_price(S_0, k, T, r, sigma, N, div_yield, div, T_div, call=0, american=1)[0]

px.line(
    x=Ns,
    y=american_option_prices,
    title='Convergence of option price',
    labels={
        'x': 'Number of steps in tree',
        'y': 'Option price'
    },
    markers=True
)

In [353]:
px.imshow(american_option_price(
    S_0=S_0,
    k=k,
    T=T,
    r=r,
    sigma=sigma,
    N=N,
    div_yield=div_yield,
    div=div,
    T_div=T_div,
    call=1,
    american=1
)[4][:, :, 1])